In [1]:
!pip install opencv-python
!pip install imutils

In [2]:
import cv2 # for working on images
import numpy as np # images are stored in numpy array
import imutils # for image processing
import argparse # giving input in command line

In [4]:
HOGCV = cv2.HOGDescriptor()

HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [5]:
def detect(frame):
  bounding_box_cordinates, weights = HOGCV.detectMultiScale(frame, winStride = (4,4), padding=(8,8), scale=1.03)
  person = 1
  for x,y,w,h in bounding_box_cordinates:
    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.putText(frame, f'person{person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    person += 1

    cv2.putText(frame, 'Status: Detecting', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Person : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0),2 )
    cv2.imshow('output', frame)

    return frame

In [6]:
def humanDetector(args):
  img_path = args["image"]
  vid_path = args["Video"]
  if str(args["Camera"]) == 'true' : Camera = True
  else: Camera = False

  writer = None
  if args['output'] is not None and img_path is None:
    writer = cv2.VideoWriter(args['output'], cv2.VideoWriter_fourcc(*'MJPG'), 10, (600, 600))

  if Camera:
        print('[INFO] Opening Web Cam.')
        detectByCamera(ouput_path,writer)
  elif vid_path is not None:
        print('[INFO] Opening Video from path.')
        detectByPathVideo(vid_path, writer)
  elif img_path is not None:
        print('[INFO] Opening Image from path.')
        detectByPathImage(img_path, args['output'])

In [7]:
def detectByCamera(writer):   # Function to detedct by camera
    video = cv2.VideoCapture(0)
    print('Detecting people...')

    while True:
        check, frame = video.read()

        frame = detect(frame)
        if writer is not None:
            writer.write(frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

In [8]:
def detectByPathVideo(path, writer): # Function to detect through video

    video = cv2.VideoCapture(path)
    check, frame = video.read()
    if check == False:
        print('Video Not Found. Please Enter a Valid Path (Full path of Video Should be Provided).')
        return

    print('Detecting people...')
    while video.isOpened():
        #check is True if reading was successful
        check, frame =  video.read()

        if check:
            frame = imutils.resize(frame , width=min(800,frame.shape[1]))
            frame = detect(frame)

            if writer is not None:
                writer.write(frame)

            key = cv2.waitKey(1)
            if key== ord('q'):
                break
        else:
            break
    video.release()
    cv2.destroyAllWindows()

def detectByCamera(writer):
    video = cv2.VideoCapture(0)
    print('Detecting people...')

    while True:
        check, frame = video.read()

        frame = detect(frame)
        if writer is not None:
            writer.write(frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
                break

    video.release()
    cv2.destroyAllWindows()

In [9]:
def detectByPathImage(path, output_path): # Function to detect through images
    image = cv2.imread(path)

    image = imutils.resize(image, width = min(800, image.shape[1]))

    result_image = detect(image)

    if output_path is not None:
        cv2.imwrite(output_path, result_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
def argsParser(): # to pass the file
    arg_parse = argparse.ArgumentParser()
    arg_parse.add_argument("-v", "--video", default=None, help="path to Video File ")
    arg_parse.add_argument("-i", "--image", default=None, help="path to Image File ")
    arg_parse.add_argument("-c", "--camera", default=True, help="Set true if you want to use the camera.")
    arg_parse.add_argument("-o", "--output", type=str, help="path to optional output video file")
    args = vars(arg_parse.parse_args())

    return args

In [ ]:
if __name__ == "__main__":
    HOGCV = cv2.HOGDescriptor()
    HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    args = argsParser()
    humanDetector(args)